In [1]:
import pandas as pd
import numpy as np
import json
import os
from os.path import join as pjoin
from cloudvolume import CloudVolume
from meshparty import skeleton

In [2]:
from utils.ccf_utils import get_ccf_ids, get_ccf_property
from utils.skeleton_loading_utils import get_skeleton, get_skeleton_ids, number_of_samples, load_lm_datasets, progress_bar

In [3]:
import platform
platstring = platform.platform()

if ('Darwin' in platstring) or ('macOS' in platstring):
    # macOS 
    data_root = "/Volumes/Brain2024/anatomy/"
elif 'Windows'  in platstring:
    # Windows (replace with the drive letter of USB drive)
    data_root = "E:/anatomy/"
elif ('amzn2' in platstring):
    # then on AWS
    data_root = "/data/"
else:
    # then your own linux platform
    # EDIT location where you mounted hard drive
    data_root = "/media/$USERNAME/Brain2024/anatomy/"

In [4]:
from brainglobe_atlasapi import BrainGlobeAtlas
import brainglobe_atlasapi as atlasapi

# Load ccf atlas
atlasapi.config.write_config_value('brainglobe_dir', f'{data_root}.brainglobe')
ccf_atlas = BrainGlobeAtlas('allen_mouse_10um')

In [5]:

LM_DATASET_KEYS = [
    "exaSPIM_609281_2022-11-03_13-49-18_reconstructions",
    "exaSPIM_651324_2023-03-06_15-13-25_reconstructions",
    "exaSPIM_653158_2023-06-01_20-41-38_reconstructions",
    "exaSPIM_653980_2023-08-10_20-08-29_reconstructions",
    "mouselight_reconstructions",
]

In [11]:
df = pd.DataFrame(columns=('genotype','skeleton_id','soma_id','soma_acronym','dendrite_ccf_ids','axon_ccf_ids','filename'))

for key in LM_DATASET_KEYS:
    path = os.path.join(data_root, key)
    print(path)
    
    #get genotype
    if key.startswith("e"):
        subject_file = os.path.join(path, "subject.json")
        print(subject_file)
        with open(subject_file, 'r') as file:
            data = json.load(file) 
        genotype = data['genotype']
    else:
        genotype = "N/A"
    
    #make dataframe
    sk_path = f"file://{data_root}{key}/precomputed"
    print(sk_path)
    cv_dataset = CloudVolume(sk_path)
    skeleton_path = os.path.join(path, "precomputed", "skeleton")
    print(skeleton_path)
    for skel_id in range(1, len(os.listdir(skeleton_path))+1):
        try:
            skel = get_skeleton(cv_dataset, skel_id)
            soma_id = int(get_ccf_ids(ccf_atlas, skel, compartment_type=1)[0])
            soma_acr = get_ccf_property(ccf_atlas, soma_id, "acronym")
            dend_ids = np.unique(get_ccf_ids(ccf_atlas, skel, compartment_type=3))
            axon_ids = np.unique(get_ccf_ids(ccf_atlas, skel, compartment_type=2))
            df.loc[len(df.index)] = [genotype, skel_id, soma_id, soma_acr, dend_ids, axon_ids, key]
        except:
            print(skel_id)
    

/data/exaSPIM_609281_2022-11-03_13-49-18_reconstructions
/data/exaSPIM_609281_2022-11-03_13-49-18_reconstructions/subject.json
file:///data/exaSPIM_609281_2022-11-03_13-49-18_reconstructions/precomputed
/data/exaSPIM_609281_2022-11-03_13-49-18_reconstructions/precomputed/skeleton
6
8
10
11
12
13
15
/data/exaSPIM_651324_2023-03-06_15-13-25_reconstructions
/data/exaSPIM_651324_2023-03-06_15-13-25_reconstructions/subject.json
file:///data/exaSPIM_651324_2023-03-06_15-13-25_reconstructions/precomputed
/data/exaSPIM_651324_2023-03-06_15-13-25_reconstructions/precomputed/skeleton
14
/data/exaSPIM_653158_2023-06-01_20-41-38_reconstructions
/data/exaSPIM_653158_2023-06-01_20-41-38_reconstructions/subject.json
file:///data/exaSPIM_653158_2023-06-01_20-41-38_reconstructions/precomputed
/data/exaSPIM_653158_2023-06-01_20-41-38_reconstructions/precomputed/skeleton
43
/data/exaSPIM_653980_2023-08-10_20-08-29_reconstructions
/data/exaSPIM_653980_2023-08-10_20-08-29_reconstructions/subject.json
file:

In [12]:
df

,genotype,skeleton_id,soma_id,soma_acronym,dendrite_ccf_ids,axon_ccf_ids,filename
0,Slc17a6-IRES-Cre/wt,1,62,mlf,"[62.0, 307.0, 354.0, 970.0, 1048.0]","[17.0, 26.0, 38.0, 42.0, 54.0, 56.0, 62.0, 63....",exaSPIM_609281_2022-11-03_13-49-18_reconstruct...
1,Slc17a6-IRES-Cre/wt,2,1048,GRN,"[136.0, 203.0, 307.0, 955.0, 1048.0, 1107.0]","[136.0, 354.0, 955.0, 1048.0, 1098.0, 1107.0, ...",exaSPIM_609281_2022-11-03_13-49-18_reconstruct...
2,Slc17a6-IRES-Cre/wt,3,437,SPVI,"[437.0, 445.0, 852.0]","[17.0, 128.0, 146.0, 206.0, 215.0, 230.0, 307....",exaSPIM_609281_2022-11-03_13-49-18_reconstruct...
3,Slc17a6-IRES-Cre/wt,4,1098,MDRNd,"[136.0, 354.0, 429.0, 955.0, 1098.0, 1107.0]","[7.0, 123.0, 136.0, 146.0, 202.0, 354.0, 437.0...",exaSPIM_609281_2022-11-03_13-49-18_reconstruct...
4,Slc17a6-IRES-Cre/wt,5,437,SPVI,[437.0],"[78.0, 96.0, 354.0, 437.0, 512.0, 728.0, 744.0...",exaSPIM_609281_2022-11-03_13-49-18_reconstruct...
...,...,...,...,...,...,...,...
1630,N/A,1545,733,VPM,"[733.0, 1020.0]","[6.0, 262.0, 657.0, 672.0, 718.0, 733.0, 806.0...",mouselight_reconstructions
1631,N/A,1546,629,VAL,"[629.0, 907.0]","[6.0, 262.0, 320.0, 577.0, 629.0, 648.0, 656.0...",mouselight_reconstructions
1632,N/A,1547,733,VPM,"[718.0, 733.0]","[9.0, 201.0, 262.0, 670.0, 672.0, 718.0, 733.0...",mouselight_reconstructions
1633,N/A,1548,382,CA1,"[382.0, 423.0, 463.0, 466.0, 10703.0]","[81.0, 250.0, 258.0, 310.0, 382.0, 423.0, 449....",mouselight_reconstructions


In [13]:
df.to_csv('/data/lm_manifest.csv')